# Execution

> FFmpeg command execution with progress tracking

In [ ]:
#| default_exp execution

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Optional, Callable
import subprocess
from tqdm.auto import tqdm

## Progress Parsing

In [ ]:
#| export
def parse_progress_line(line: str # A line of stderr output from ffmpeg
                        ) -> Optional[float]: # Current time in seconds, or None if line doesn't contain progress info
    """
    Parse a progress line from ffmpeg stderr output.
    """
    # Look for time progress in format "out_time_ms=123456789"
    if line.startswith('out_time_ms='):
        try:
            time_ms = int(line.strip().split('=')[1])
            return time_ms / 1000000.0  # Convert microseconds to seconds
        except (ValueError, IndexError):
            pass
    
    # Alternative: look for time in format "time=00:01:23.45"
    elif line.startswith('time='):
        try:
            time_str = line.strip().split('=')[1]
            # Parse time format HH:MM:SS.ss
            time_parts = time_str.split(':')
            if len(time_parts) == 3:
                hours = float(time_parts[0])
                minutes = float(time_parts[1])
                seconds = float(time_parts[2])
                return hours * 3600 + minutes * 60 + seconds
        except (ValueError, IndexError):
            pass
    
    return None

## FFmpeg Execution with Progress

In [ ]:
#| export
def run_ffmpeg_with_progress(
    cmd: List[str], # List containing the ffmpeg command and arguments
    total_duration: Optional[float] = None, # Total duration in seconds for determinate progress, or None for indeterminate progress
    description: str = "Processing", # Description text for the progress bar
    verbose: bool = False, # If True, shows detailed ffmpeg output
    progress_callback: Optional[Callable[[float], None]] = None # Optional callback function that receives current progress in seconds as an argument
) -> None: # Raises FileNotFoundError if ffmpeg is not installed/found, or subprocess.CalledProcessError if ffmpeg command fails
    """
    Run an ffmpeg command with a progress bar.
    <br>
    Raises:<br>
    - FileNotFoundError: If input file doesn't exist<br>
    - subprocess.CalledProcessError: If ffmpeg command fails<br>
    - FileNotFoundError: If ffmpeg is not installed/found<br>    
    """
    try:
        # Initialize progress bar
        if total_duration:
            pbar = tqdm(
                total=total_duration, 
                unit='s', 
                desc=description,
                bar_format='{l_bar}{bar}| {n:.1f}/{total:.1f}s [{elapsed}<{remaining}]'
            )
        else:
            pbar = tqdm(desc=description, unit='frames')
        
        # Run ffmpeg command with real-time progress
        process = subprocess.Popen(
            cmd,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        # Parse progress from stderr
        while process.poll() is None:  # Continue while process is running
            line = process.stderr.readline()
            if not line:
                continue
                
            if verbose:
                print(line.strip())
            
            current_time = parse_progress_line(line)
            if current_time is not None:
                if total_duration:
                    pbar.n = min(current_time, total_duration)
                    pbar.refresh()
                else:
                    pbar.update(1)
                
                # Call progress callback if provided
                if progress_callback:
                    progress_callback(current_time)
        
        # Process any remaining stderr output
        remaining_stderr = process.stderr.read()
        if verbose and remaining_stderr:
            print(remaining_stderr)
        
        # Get the return code
        return_code = process.returncode
        
        # Ensure progress bar reaches 100% when done
        if total_duration:
            pbar.n = total_duration
            pbar.refresh()
        
        pbar.close()
        
        if return_code != 0:
            # Get any error output
            stderr_output = remaining_stderr or ""
            error_msg = f"FFmpeg failed with return code {return_code}"
            if stderr_output:
                error_msg += f"\nError output: {stderr_output}"
            raise subprocess.CalledProcessError(return_code, cmd, error_msg)
            
    except FileNotFoundError:
        if 'pbar' in locals():
            pbar.close()
        raise FileNotFoundError(
            "ffmpeg not found. Please install ffmpeg and ensure it's in your PATH."
        )
    except Exception as e:
        if 'pbar' in locals():
            pbar.close()
        raise

## Testing

In [ ]:
#|eval: false
# Test progress line parsing
test_line1 = "out_time_ms=5000000"
assert parse_progress_line(test_line1) == 5.0

test_line2 = "time=00:01:30.00"
assert parse_progress_line(test_line2) == 90.0

test_line3 = "random line"
assert parse_progress_line(test_line3) is None

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()